# Preparação do ambiente

## Instalação de dependências

In [6]:
# !pip install pandas
# !pip install python-dotenv
# !pip install requests
# !pip install openai==0.28.1

## Importação das bibliotecas

In [7]:
# Importação das bibliotecas
import os
import openai
from dotenv import load_dotenv

## Carregar variáveis de ambiente

In [8]:
load_dotenv()

True

## Configuração do OpenRouter

In [9]:
openai.api_key = os.getenv("OPENROUTER_API_KEY")
openai.api_base = "https://openrouter.ai/api/v1"

In [10]:
# Função para gerar resposta do modelo
def get_chatbot_response(messages, system_prompt="", temperature=0):
    response = openai.ChatCompletion.create(
        model="meta-llama/llama-3.1-8b-instruct:free",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": messages}
        ],
        temperature=temperature
    )

    return response["choices"][0]["message"]["content"]

In [11]:
# Teste da função
msg_usuario = "Qual é a capital da Austrália?"
resposta = get_chatbot_response(msg_usuario)
print(f"Usuário: {msg_usuario}\nChatBot: {resposta}")

Usuário: Qual é a capital da Austrália?
ChatBot: A capital da Austrália é Canberra.
